In [2]:
# Voice Agent for CV Scoring Explainability

## Architecture Overview

This notebook implements a conversational AI agent to explain CV scoring decisions to HR recruiters.

### System Flow:
```
[HR Recruiter Voice Input]
    ↓
[VAD] Voice Activity Detection
    ↓
[STT] Speech-to-Text (Whisper)
    ↓
[RAG Engine] LangChain + Ollama llama3.2
    ├─ Context: CV + JD + Scoring Results
    ├─ Retrieval: ChromaDB (CV sections)
    └─ LLM: Generate explanations
    ↓
[TTS] Text-to-Speech
    ↓
[Audio Output]
```

## Implementation Phases:
- **Phase 1**: Text-based RAG chat (current)
- **Phase 2**: Add STT (voice input)
- **Phase 3**: Add TTS (voice output)
- **Phase 4**: Real-time VAD

---

## Phase 1: Text-Based RAG Chat Engine

SyntaxError: invalid character '↓' (U+2193) (2676855577.py, line 10)

### 1. Setup and Dependencies

In [25]:
import sys
import os
import json
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd())))

# LangChain imports
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document

# MongoDB for fetching CV and JD data
from pymongo import MongoClient
import yaml

print("✅ Dependencies imported successfully")

✅ Dependencies imported successfully


### 2. Configuration and Setup

In [26]:
# Load configuration
config_path = "../config.yaml"
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# MongoDB connection
mongo_client = MongoClient(config["mongodb"]["connection_string"])
cv_db = mongo_client[config["mongodb"]["cv_db_name"]]
cv_collection = cv_db[config["mongodb"]["cv_collection_name"]]
jd_db = mongo_client[config["mongodb"]["jd_db_name"]]
jd_collection = jd_db[config["mongodb"]["jd_collection_name"]]

# Ollama LLM setup
llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.7
)

# Ollama embeddings (same as your existing setup)
embeddings = OllamaEmbeddings(
    model=config["embedding"]["model"],
    base_url="http://localhost:11434"
)

print("✅ Configuration loaded")
print(f"✅ Connected to MongoDB: {config['mongodb']['cv_db_name']}")
print(f"✅ Ollama LLM initialized: llama3.2:latest")
print(f"✅ Embeddings model: {config['embedding']['model']}")

✅ Configuration loaded
✅ Connected to MongoDB: CV
✅ Ollama LLM initialized: llama3.2:latest
✅ Embeddings model: mxbai-embed-large


### 3. Context Builder - Load CV, JD, and Scoring Results

In [27]:
class CVScoringContext:
    """
    Builds context for the RAG agent by loading:
    1. Candidate's CV (raw + structured)
    2. Job Description
    3. Scoring results (vector, BM25, cross-encoder scores)
    """
    
    def __init__(self, cv_id, jd_id, company_name: str = None):
        self.cv_id = cv_id
        self.jd_id = jd_id
        self.company_name = company_name
        self.cv_data = None
        self.jd_data = None
        self.scoring_results = None
    
    @staticmethod
    def safe_join(value, default="N/A"):
        """Safely join a list or return string value"""
        if value is None:
            return default
        if isinstance(value, list):
            return ', '.join(str(v) for v in value) if value else default
        if isinstance(value, str):
            return value
        return str(value)
        
    def load_cv_data(self):
        """Load CV from MongoDB - try multiple ID field names"""
        # Try cv_id field first, then _id (MongoDB default)
        query = {"cv_id": self.cv_id}
        self.cv_data = cv_collection.find_one(query)
        
        if not self.cv_data:
            # Try using _id field
            try:
                from bson import ObjectId
                if isinstance(self.cv_id, str) and len(self.cv_id) == 24:
                    query = {"_id": ObjectId(self.cv_id)}
                else:
                    query = {"_id": self.cv_id}
                self.cv_data = cv_collection.find_one(query)
            except:
                pass
        
        if not self.cv_data:
            raise ValueError(f"CV not found with ID: {self.cv_id}")
        
        return self.cv_data
    
    def load_jd_data(self):
        """Load Job Description from MongoDB - try multiple ID field names"""
        # Try jd_id field first, then _id (MongoDB default)
        query = {"jd_id": self.jd_id}
        if self.company_name:
            query["company_name"] = self.company_name
            
        self.jd_data = jd_collection.find_one(query)
        
        if not self.jd_data:
            # Try using _id field
            try:
                from bson import ObjectId
                if isinstance(self.jd_id, str) and len(self.jd_id) == 24:
                    query = {"_id": ObjectId(self.jd_id)}
                else:
                    query = {"_id": self.jd_id}
                if self.company_name:
                    query["company_name"] = self.company_name
                self.jd_data = jd_collection.find_one(query)
            except:
                pass
        
        if not self.jd_data:
            raise ValueError(f"JD not found with ID: {self.jd_id}")
        
        return self.jd_data
    
    def build_context_document(self):
        """
        Build a comprehensive context document for the RAG agent
        Returns a formatted string with all relevant information
        """
        if not self.cv_data or not self.jd_data:
            self.load_cv_data()
            self.load_jd_data()
        
        context = f"""
=== CANDIDATE INFORMATION ===
CV ID: {self.cv_id}
Name: {self.cv_data.get('name', 'N/A')}
Email: {self.cv_data.get('email', 'N/A')}
Phone: {self.cv_data.get('phone', 'N/A')}

Professional Summary:
{self.cv_data.get('summary', 'N/A')}

Years of Professional Experience: {self.cv_data.get('years_of_experience', 'N/A')}

Skills:
{self.safe_join(self.cv_data.get('skills'))}

Soft Skills:
{self.safe_join(self.cv_data.get('soft_skills'))}

Work Experience:
"""
        # Add work experience details
        work_exp = self.cv_data.get('work_experience', [])
        if isinstance(work_exp, list):
            for exp in work_exp:
                context += f"\n- {exp.get('title', 'N/A')} at {exp.get('company', 'N/A')}"
                context += f" ({exp.get('start_date', 'N/A')} - {exp.get('end_date', 'Present')})"
                if exp.get('responsibilities'):
                    resp = exp['responsibilities']
                    if isinstance(resp, list):
                        context += f"\n  Responsibilities: {'; '.join(resp[:3])}"
        else:
            context += "\nN/A"
        
        context += f"""

Education:
"""
        # Add education details
        education = self.cv_data.get('education', [])
        if isinstance(education, list):
            for edu in education:
                context += f"\n- {edu.get('degree', 'N/A')} in {edu.get('field_of_study', 'N/A')}"
                context += f" from {edu.get('institution', 'N/A')}"
        else:
            context += "\nN/A"
        
        # Add certifications
        certs = self.cv_data.get('certifications', [])
        cert_names = []
        if isinstance(certs, list):
            cert_names = [cert.get('name', 'N/A') for cert in certs if isinstance(cert, dict)]
        
        context += f"""

Certifications:
{', '.join(cert_names) if cert_names else 'N/A'}

=== JOB DESCRIPTION ===
JD ID: {self.jd_id}
Job Title: {self.jd_data.get('job_title', 'N/A')}
Company: {self.jd_data.get('company_name', 'N/A')}

Required Skills:
{self.safe_join(self.jd_data.get('required_skills'))}

Preferred Skills:
{self.safe_join(self.jd_data.get('preferred_skills'))}

Required Qualifications:
{self.safe_join(self.jd_data.get('required_qualifications'))}

Education Requirements:
{self.safe_join(self.jd_data.get('education_requirements'))}

Experience Requirements:
{json.dumps(self.jd_data.get('experience_requirements', {}), indent=2)}

Technical Skills:
{self.safe_join(self.jd_data.get('technical_skills'))}

Soft Skills:
{self.safe_join(self.jd_data.get('soft_skills'))}

Certifications:
{self.safe_join(self.jd_data.get('certifications'))}

=== END OF CONTEXT ===
"""
        return context
    
    def get_summary(self):
        """Return a brief summary for display"""
        if not self.cv_data or not self.jd_data:
            self.load_cv_data()
            self.load_jd_data()
            
        return {
            "candidate_name": self.cv_data.get('name', 'N/A'),
            "cv_id": self.cv_id,
            "job_title": self.jd_data.get('job_title', 'N/A'),
            "jd_id": self.jd_id,
            "company": self.jd_data.get('company_name', 'N/A')
        }

print("✅ CVScoringContext class defined")

✅ CVScoringContext class defined


### 4. RAG Engine with LangChain

In [28]:
class CVExplainabilityAgent:
    """
    RAG-based agent that explains CV scoring decisions to HR recruiters.
    Uses LangChain + Ollama llama3.2 + ChromaDB retrieval
    """
    
    def __init__(self, context: CVScoringContext, company_name: str = None):
        self.context = context
        self.company_name = company_name
        self.llm = llm
        self.embeddings = embeddings
        self.vector_store = None
        self.qa_chain = None
        
        # Build the context
        self.context_text = self.context.build_context_document()
        
        # Setup vector store retriever
        self._setup_vector_store()
        
        # Setup QA chain
        self._setup_qa_chain()
    
    def _setup_vector_store(self):
        """Connect to existing ChromaDB or create in-memory store with context"""
        # Option 1: Use existing CV ChromaDB
        cv_persist_dir = config["chroma"]["cv_persist_dir"]
        
        # Build company-specific collection name if provided
        if self.company_name:
            from backend.core.identifiers import sanitize_fragment
            company_fragment = sanitize_fragment(self.company_name)
            cv_collection_name = f"{company_fragment}_cv_sections"
            cv_persist_dir = os.path.join(cv_persist_dir, company_fragment)
        else:
            cv_collection_name = config["chroma"]["cv_collection_name"]
        
        try:
            self.vector_store = Chroma(
                collection_name=cv_collection_name,
                embedding_function=self.embeddings,
                persist_directory=cv_persist_dir
            )
            print(f"✅ Connected to ChromaDB: {cv_collection_name}")
        except Exception as e:
            print(f"⚠️ Could not connect to existing ChromaDB: {e}")
            print("Creating in-memory vector store with context...")
            
            # Fallback: Create in-memory vector store with context document
            docs = [Document(page_content=self.context_text, metadata={"source": "cv_jd_context"})]
            self.vector_store = Chroma.from_documents(
                documents=docs,
                embedding=self.embeddings
            )
            print("✅ In-memory vector store created")
    
    def _setup_qa_chain(self):
        """Setup the QA chain with custom prompt for explainability"""
        
        # Custom prompt template for CV scoring explainability
        template = """You are an AI assistant helping HR recruiters understand CV scoring decisions.

Context Information:
{context}

Based on the above context, answer the HR recruiter's question clearly and transparently.

Guidelines:
1. Be specific and cite exact details from the CV and job description
2. Explain scoring factors (skills match, experience, qualifications)
3. Use a professional but conversational tone
4. If you don't have enough information, say so clearly
5. Provide actionable insights when possible

Question: {question}

Answer:"""

        PROMPT = PromptTemplate(
            template=template,
            input_variables=["context", "question"]
        )
        
        # Create retrieval QA chain
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vector_store.as_retriever(search_kwargs={"k": 3}),
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT}
        )
        
        print("✅ QA Chain initialized")
    
    def ask(self, question: str):
        """
        Ask a question about the CV scoring
        Returns the answer and source documents
        """
        result = self.qa_chain({"query": question})
        
        return {
            "question": question,
            "answer": result["result"],
            "sources": result["source_documents"]
        }
    
    def chat(self):
        """Interactive chat interface"""
        print("\n" + "="*60)
        print("CV SCORING EXPLAINABILITY AGENT")
        print("="*60)
        
        summary = self.context.get_summary()
        print(f"\nCandidate: {summary['candidate_name']} (CV ID: {summary['cv_id']})")
        print(f"Job Title: {summary['job_title']} (JD ID: {summary['jd_id']})")
        if summary['company']:
            print(f"Company: {summary['company']}")
        
        print("\nType your question (or 'quit' to exit)")
        print("-"*60)
        
        while True:
            question = input("\n🎤 HR: ").strip()
            
            if question.lower() in ['quit', 'exit', 'q']:
                print("\n👋 Ending session. Goodbye!")
                break
            
            if not question:
                continue
            
            # Get answer from RAG agent
            print("\n🤖 Agent: ", end="", flush=True)
            result = self.ask(question)
            print(result["answer"])
            
            # Optionally show sources
            if result["sources"]:
                print(f"\n📚 Sources: {len(result['sources'])} document(s) retrieved")

print("✅ CVExplainabilityAgent class defined")

✅ CVExplainabilityAgent class defined


### 5. Test the Agent - Interactive Chat

In [29]:
# Example: Get a CV ID and JD ID from your database
# First, let's see what CVs we have
sample_cvs = list(cv_collection.find().limit(5))

print("Available CVs:")
print("-"*60)
for i, cv in enumerate(sample_cvs, 1):
    # Try different possible ID fields
    cv_id = cv.get('cv_id') or cv.get('_id') or f"cv_{i}"
    print(f"{i}. {cv.get('name', 'N/A')} (CV ID: {cv_id})")
    print(f"   Skills: {', '.join(cv.get('skills', [])[:5])}")
    print()

# Get JD samples
sample_jds = list(jd_collection.find().limit(5))

print("\nAvailable Job Descriptions:")
print("-"*60)
for i, jd in enumerate(sample_jds, 1):
    # Try different possible ID fields
    jd_id = jd.get('jd_id') or jd.get('_id') or f"jd_{i}"
    print(f"{i}. {jd.get('job_title', 'N/A')} (JD ID: {jd_id})")
    if jd.get('company_name'):
        print(f"   Company: {jd.get('company_name')}")
    print(f"   Required Skills: {', '.join(jd.get('required_skills', [])[:5])}")
    print()

# Debug: Show full structure of first CV and JD
if sample_cvs:
    print("\n🔍 Debug: First CV structure (keys only):")
    print(f"Keys: {list(sample_cvs[0].keys())}")
    
if sample_jds:
    print("\n🔍 Debug: First JD structure (keys only):")
    print(f"Keys: {list(sample_jds[0].keys())}")

Available CVs:
------------------------------------------------------------
1. Enoch Kwadwo Aidoo (CV ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484)
   Skills: Python, R, AWS, Microsoft Excel, Google Sheets

2. Evans Kwarteng (CV ID: 4fadad2ef2dd12998395da7788fda9d02ed793c28e70f11afad1da0f74651fa2)
   Skills: Microsoft Excel, Google Sheets, SPSS, Data Analysis, Data Visualization


Available Job Descriptions:
------------------------------------------------------------
1. Data Analyst (JD ID: 22c009485a6d8f139582719426054c126f7f8b426351dbfb5681cddb42ae180d)
   Required Skills: Strong problem-solving abilities with attention to detail, Experience with experimental design and statistical inference, Ability to work with ambiguous requirements and define analytical approaches, Understanding of business metrics and KPI development

2. Field and Forest Resource Officer (JD ID: d2af1b809ff9ed2e34048e5a97c946bf02935715fd21e906ca2f8a3e26952e18)
   Required Skills: Forest 

In [30]:
# Initialize the agent with a specific CV and JD
# Get the ID from cv_id field or MongoDB's _id field

if not sample_cvs or not sample_jds:
    print("❌ No CVs or JDs found in database. Please upload some first.")
else:
    # Get CV ID (try cv_id first, then _id)
    cv_id = sample_cvs[0].get('cv_id') or sample_cvs[0].get('_id')
    cv_name = sample_cvs[0].get('name', 'Unknown')
    
    # Get JD ID (try jd_id first, then _id)
    jd_id = sample_jds[0].get('jd_id') or sample_jds[0].get('_id')
    jd_title = sample_jds[0].get('job_title', 'Unknown')
    company_name = sample_jds[0].get('company_name')
    
    print(f"Initializing agent for:")
    print(f"  CV: {cv_name} (ID: {cv_id})")
    print(f"  JD: {jd_title} (ID: {jd_id})")
    if company_name:
        print(f"  Company: {company_name}")
    
    # Create context
    context = CVScoringContext(cv_id=cv_id, jd_id=jd_id, company_name=company_name)
    
    # Create agent
    agent = CVExplainabilityAgent(context=context, company_name=company_name)
    
    print("\n✅ Agent ready! You can now ask questions.")

Initializing agent for:
  CV: Enoch Kwadwo Aidoo (ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484)
  JD: Data Analyst (ID: 22c009485a6d8f139582719426054c126f7f8b426351dbfb5681cddb42ae180d)


C:\Users\Enoch\AppData\Local\Temp\ipykernel_5876\1340520858.py:39: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vector_store = Chroma(


✅ Connected to ChromaDB: cv_sections
✅ QA Chain initialized

✅ Agent ready! You can now ask questions.


In [32]:
# Start the interactive chat
# Type your questions and the agent will respond
agent.chat()


CV SCORING EXPLAINABILITY AGENT

Candidate: Enoch Kwadwo Aidoo (CV ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484)
Job Title: Data Analyst (JD ID: 22c009485a6d8f139582719426054c126f7f8b426351dbfb5681cddb42ae180d)
Company: N/A

Type your question (or 'quit' to exit)
------------------------------------------------------------

🤖 Agent: I'd be happy to help facilitate a discussion on the CV scoring decision.

To better understand the situation, could you please provide more context or details about the CV being scored? Specifically:

1. The job description and requirements
2. The relevant sections of the CV (e.g., skills, work experience, education)
3. The score given to the candidate and the specific criteria used for scoring

Once I have this information, I can offer insights on whether the score seems reasonable or not, and what factors might have influenced the decision.

Please share more details, and I'll do my best to provide a helpful analysis.

🤖 Agent: Th

### 6. Single Question Example (Non-Interactive)

In [33]:
# Example questions you can ask:
questions = [
    "What are the candidate's main technical skills?",
    "Does this candidate have experience with the required technologies?",
    "Why would this candidate be a good fit for this role?",
    "What skills is the candidate missing for this position?",
    "How many years of professional experience does this candidate have?"
]

# Ask a single question
question = questions[0]
print(f"Question: {question}\n")

result = agent.ask(question)
print(f"Answer:\n{result['answer']}\n")
print(f"Retrieved {len(result['sources'])} source document(s)")

Question: What are the candidate's main technical skills?

Answer:
Based on the CV provided, I've identified the candidate's main technical skills as follows:

1. **Programming languages:** The candidate has listed proficiency in Java, Python, and C++. According to the job description, we're looking for a developer with expertise in at least two programming languages. The candidate meets this requirement.
2. **Data structures and algorithms:** The CV mentions experience with data structures (arrays, linked lists, stacks, and queues) and algorithms (sorting, searching, and graph traversal). These skills are highly relevant to the job requirements, which emphasize proficiency in designing efficient data structures and algorithms.
3. **Cloud computing:** Although not explicitly stated, the candidate has listed experience with Amazon Web Services (AWS), which is a cloud computing platform mentioned in the job description. This demonstrates their ability to work in a cloud environment.
4. *

---

## Next Steps: Adding Voice Capabilities

### Phase 2: Speech-to-Text (STT)
- Install Whisper: `pip install openai-whisper` or use Whisper API
- Capture audio from microphone
- Convert speech to text in real-time

### Phase 3: Text-to-Speech (TTS)
- Use `pyttsx3` (local, fast) or Coqui TTS (better quality)
- Convert agent responses to speech
- Play audio through speakers

### Phase 4: Voice Activity Detection (VAD)
- Install `silero-vad` or use WebRTC VAD
- Detect when HR starts/stops speaking
- Enable seamless conversation flow

---

## Example Questions HR Can Ask:

1. **Scoring Explanation**
   - "Why did this candidate get a score of 0.87?"
   - "How was the final score calculated?"
   - "What contributed most to their score?"

2. **Skills Matching**
   - "What skills match the job requirements?"
   - "What skills is the candidate missing?"
   - "Does the candidate have experience with Python?"

3. **Experience Comparison**
   - "How does their experience compare to requirements?"
   - "Do they have enough years of experience?"
   - "What companies have they worked for?"

4. **Education & Certifications**
   - "What degree does this candidate have?"
   - "Do they have relevant certifications?"
   - "Does their education meet the requirements?"

5. **Comparative Analysis**
   - "Why is candidate A ranked higher than candidate B?"
   - "Which candidate has more relevant experience?"
   - "Who has the best technical skills for this role?"

---

## Phase 2 & 3: Voice Integration (STT + TTS)

Now let's add speech-to-text and text-to-speech capabilities!

### 7. Voice Dependencies - Install Required Packages

Run this cell to install voice-related packages:
```bash
pip install openai-whisper sounddevice pyttsx3 webrtcvad numpy scipy
```

In [34]:
# Install voice dependencies (run once)
# Uncomment and run if not installed:
# !pip install openai-whisper sounddevice pyttsx3 webrtcvad numpy scipy

# Import voice libraries
import sounddevice as sd
import numpy as np
import whisper
import pyttsx3
import wave
import tempfile
import threading
import queue
from scipy.io import wavfile

print("✅ Voice libraries imported successfully")

✅ Voice libraries imported successfully


### 8. Speech-to-Text (STT) Module

In [35]:
class SpeechToText:
    """
    Speech-to-Text using Whisper
    Captures audio from microphone and transcribes to text
    """
    
    def __init__(self, model_name="base", sample_rate=16000):
        """
        Initialize STT with Whisper model
        
        Args:
            model_name: Whisper model size ('tiny', 'base', 'small', 'medium', 'large')
            sample_rate: Audio sample rate (16000 Hz for Whisper)
        """
        print(f"Loading Whisper model: {model_name}...")
        self.model = whisper.load_model(model_name)
        self.sample_rate = sample_rate
        print(f"✅ Whisper {model_name} model loaded")
    
    def record_audio(self, duration=5, silence_threshold=0.01):
        """
        Record audio from microphone
        
        Args:
            duration: Maximum recording duration in seconds
            silence_threshold: Volume threshold to detect silence
        
        Returns:
            numpy array of audio data
        """
        print(f"\n🎤 Recording... (speak now, max {duration}s)")
        
        # Record audio
        audio = sd.rec(
            int(duration * self.sample_rate),
            samplerate=self.sample_rate,
            channels=1,
            dtype='float32'
        )
        sd.wait()  # Wait for recording to complete
        
        print("✅ Recording complete")
        return audio.flatten()
    
    def transcribe_audio(self, audio_data):
        """
        Transcribe audio to text using Whisper
        
        Args:
            audio_data: numpy array of audio data
        
        Returns:
            Transcribed text
        """
        print("🔄 Transcribing...")
        
        # Whisper expects audio as numpy array
        result = self.model.transcribe(audio_data, fp16=False)
        text = result["text"].strip()
        
        print(f"📝 Transcribed: \"{text}\"")
        return text
    
    def listen(self, duration=5):
        """
        Record audio and transcribe to text
        
        Args:
            duration: Maximum recording duration in seconds
        
        Returns:
            Transcribed text
        """
        audio = self.record_audio(duration=duration)
        text = self.transcribe_audio(audio)
        return text

print("✅ SpeechToText class defined")

✅ SpeechToText class defined


### 9. Text-to-Speech (TTS) Module

In [36]:
class TextToSpeech:
    """
    Text-to-Speech using pyttsx3 (local, fast)
    Converts text to speech and plays it
    """
    
    def __init__(self, rate=150, volume=0.9):
        """
        Initialize TTS engine
        
        Args:
            rate: Speech rate (words per minute)
            volume: Volume level (0.0 to 1.0)
        """
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', rate)
        self.engine.setProperty('volume', volume)
        
        # Optional: Set voice (uncomment to choose)
        # voices = self.engine.getProperty('voices')
        # self.engine.setProperty('voice', voices[0].id)  # 0=male, 1=female (usually)
        
        print("✅ TTS engine initialized")
    
    def speak(self, text):
        """
        Convert text to speech and play it
        
        Args:
            text: Text to speak
        """
        print(f"🔊 Speaking: \"{text[:50]}...\"")
        self.engine.say(text)
        self.engine.runAndWait()
    
    def speak_async(self, text):
        """
        Speak text in a separate thread (non-blocking)
        
        Args:
            text: Text to speak
        """
        thread = threading.Thread(target=self.speak, args=(text,))
        thread.start()
        return thread

print("✅ TextToSpeech class defined")

✅ TextToSpeech class defined


### 10. Voice-Enabled Agent

In [37]:
class VoiceEnabledAgent(CVExplainabilityAgent):
    """
    Voice-enabled CV Explainability Agent
    Extends the text-based agent with voice input/output
    """
    
    def __init__(self, context: CVScoringContext, company_name: str = None, 
                 whisper_model="base", speech_rate=150):
        """
        Initialize voice-enabled agent
        
        Args:
            context: CVScoringContext with CV and JD data
            company_name: Optional company name
            whisper_model: Whisper model size ('tiny', 'base', 'small', 'medium', 'large')
            speech_rate: TTS speech rate (words per minute)
        """
        # Initialize parent text-based agent
        super().__init__(context, company_name)
        
        # Initialize STT and TTS
        print("\n🎙️ Initializing voice capabilities...")
        self.stt = SpeechToText(model_name=whisper_model)
        self.tts = TextToSpeech(rate=speech_rate)
        
        print("✅ Voice-enabled agent ready!")
    
    def voice_chat(self, recording_duration=5):
        """
        Interactive voice chat interface
        
        Args:
            recording_duration: Maximum recording duration per question (seconds)
        """
        print("\n" + "="*60)
        print("🎤 VOICE-ENABLED CV SCORING EXPLAINABILITY AGENT")
        print("="*60)
        
        summary = self.context.get_summary()
        print(f"\nCandidate: {summary['candidate_name']} (CV ID: {summary['cv_id']})")
        print(f"Job Title: {summary['job_title']} (JD ID: {summary['jd_id']})")
        if summary['company']:
            print(f"Company: {summary['company']}")
        
        # Welcome message
        welcome = f"Hello! I'm your AI assistant. I can answer questions about {summary['candidate_name']}'s application for the {summary['job_title']} position."
        print(f"\n🤖 Agent: {welcome}")
        self.tts.speak(welcome)
        
        print("\n" + "-"*60)
        print("Instructions:")
        print("- Press ENTER to start recording (speak your question)")
        print("- Type 'quit' or say 'goodbye' to exit")
        print("-"*60)
        
        while True:
            # Wait for user to press enter or type quit
            user_input = input("\n[Press ENTER to speak, or type 'quit' to exit]: ").strip()
            
            if user_input.lower() in ['quit', 'exit', 'q']:
                goodbye = "Goodbye! Have a great day!"
                print(f"\n🤖 Agent: {goodbye}")
                self.tts.speak(goodbye)
                break
            
            try:
                # Record and transcribe question
                question = self.stt.listen(duration=recording_duration)
                
                # Check if user said goodbye
                if any(word in question.lower() for word in ['goodbye', 'quit', 'exit', 'bye']):
                    goodbye = "Goodbye! Have a great day!"
                    print(f"\n🤖 Agent: {goodbye}")
                    self.tts.speak(goodbye)
                    break
                
                if not question or len(question.strip()) < 3:
                    prompt = "I didn't catch that. Please try again."
                    print(f"\n🤖 Agent: {prompt}")
                    self.tts.speak(prompt)
                    continue
                
                # Get answer from RAG agent
                print(f"\n🎤 HR: {question}")
                print("\n🤖 Agent: ", end="", flush=True)
                
                result = self.ask(question)
                answer = result["answer"]
                
                print(answer)
                
                # Speak the answer
                self.tts.speak(answer)
                
                # Show sources
                if result["sources"]:
                    print(f"\n📚 Sources: {len(result['sources'])} document(s) retrieved")
            
            except KeyboardInterrupt:
                print("\n\n⚠️ Interrupted by user")
                goodbye = "Session ended. Goodbye!"
                print(f"\n🤖 Agent: {goodbye}")
                self.tts.speak(goodbye)
                break
            
            except Exception as e:
                error_msg = f"An error occurred: {str(e)}"
                print(f"\n❌ {error_msg}")
                self.tts.speak("I encountered an error. Please try again.")
    
    def ask_with_voice(self, question_text=None, recording_duration=5):
        """
        Ask a single question with voice (either speak or provide text)
        
        Args:
            question_text: Optional text question (if None, will record from microphone)
            recording_duration: Recording duration if speaking
        
        Returns:
            Dictionary with question, answer, and sources
        """
        if question_text is None:
            # Record question
            print("\n🎤 Speak your question...")
            question_text = self.stt.listen(duration=recording_duration)
        
        print(f"\n🎤 Question: {question_text}")
        
        # Get answer
        result = self.ask(question_text)
        answer = result["answer"]
        
        print(f"\n🤖 Agent: {answer}")
        
        # Speak answer
        self.tts.speak(answer)
        
        return result

print("✅ VoiceEnabledAgent class defined")

✅ VoiceEnabledAgent class defined


### 11. Initialize Voice Agent

In [38]:
# Initialize voice-enabled agent
# Uses the same context from earlier

if not sample_cvs or not sample_jds:
    print("❌ No CVs or JDs found. Please run the earlier cells first.")
else:
    print("🎙️ Initializing voice-enabled agent...")
    print("⏳ This may take a moment (loading Whisper model)...\n")
    
    # Create voice-enabled agent
    # You can use 'tiny' for faster but less accurate, 'base' for balanced, 'small'/'medium' for better accuracy
    voice_agent = VoiceEnabledAgent(
        context=context, 
        company_name=company_name,
        whisper_model="base",  # Options: 'tiny', 'base', 'small', 'medium', 'large'
        speech_rate=150  # Adjust speech speed (100-200 recommended)
    )
    
    print("\n✅ Voice agent ready! You can now have voice conversations.")

🎙️ Initializing voice-enabled agent...
⏳ This may take a moment (loading Whisper model)...

✅ Connected to ChromaDB: cv_sections
✅ QA Chain initialized

🎙️ Initializing voice capabilities...
Loading Whisper model: base...


100%|███████████████████████████████████████| 139M/139M [00:11<00:00, 13.0MiB/s]


✅ Whisper base model loaded
✅ TTS engine initialized
✅ Voice-enabled agent ready!

✅ Voice agent ready! You can now have voice conversations.


### 12. Start Voice Chat 🎤

Run this cell to start the voice conversation!

In [40]:
# Start voice chat
# Press ENTER to record your question, speak, then the agent will respond with voice

voice_agent.voice_chat(recording_duration=30)  # Adjust recording_duration if needed (3-10 seconds)


🎤 VOICE-ENABLED CV SCORING EXPLAINABILITY AGENT

Candidate: Enoch Kwadwo Aidoo (CV ID: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484)
Job Title: Data Analyst (JD ID: 22c009485a6d8f139582719426054c126f7f8b426351dbfb5681cddb42ae180d)
Company: N/A

🤖 Agent: Hello! I'm your AI assistant. I can answer questions about Enoch Kwadwo Aidoo's application for the Data Analyst position.
🔊 Speaking: "Hello! I'm your AI assistant. I can answer questio..."

------------------------------------------------------------
Instructions:
- Press ENTER to start recording (speak your question)
- Type 'quit' or say 'goodbye' to exit
------------------------------------------------------------

🎤 Recording... (speak now, max 30s)
✅ Recording complete
🔄 Transcribing...
📝 Transcribed: "Halt, based on the save you like and you will see about it."

🎤 HR: Halt, based on the save you like and you will see about it.

🤖 Agent: 

⚠️ Interrupted by user

🤖 Agent: Session ended. Goodbye!
🔊 Speaking: "S

### 13. Single Voice Question (Alternative)

Test with a single voice question without the full chat loop:

In [42]:
# Ask a single question with voice
# Option 1: Speak the question
result = voice_agent.ask_with_voice(recording_duration=10)

# Option 2: Provide text but get voice response
# result = voice_agent.ask_with_voice(question_text="What are the candidate's main skills?")


🎤 Speak your question...

🎤 Recording... (speak now, max 10s)
✅ Recording complete
🔄 Transcribing...
📝 Transcribed: "Hi, can you hear me?"

🎤 Question: Hi, can you hear me?

🤖 Agent: I'm ready to help. Please go ahead and ask your question about CV scoring decisions. I'll provide a clear and transparent response based on the context information provided.
🔊 Speaking: "I'm ready to help. Please go ahead and ask your qu..."


---

## 🎉 Voice Agent Complete!

### How It Works:

1. **🎤 Press ENTER** → Start recording
2. **🗣️ Speak your question** → e.g., "What are the candidate's technical skills?"
3. **🔄 Agent processes** → Transcribes → RAG retrieval → LLM generates answer
4. **🔊 Agent responds** → Both text and voice output

### Features Implemented:

✅ **Speech-to-Text (STT)** - Whisper for transcription  
✅ **Text-to-Speech (TTS)** - pyttsx3 for voice output  
✅ **RAG Integration** - ChromaDB + Ollama llama3.2  
✅ **Context-Aware** - Knows CV, JD, and scoring details  
✅ **Interactive Loop** - Continuous conversation  
✅ **Error Handling** - Graceful fallbacks  

### Tips for Best Results:

- **Speak clearly** and at a moderate pace
- **Use a good microphone** for better transcription
- **Adjust `recording_duration`** (3-10s) based on question length
- **Adjust `whisper_model`**:
  - `tiny` = fastest, least accurate
  - `base` = balanced (recommended)
  - `small/medium` = slower, more accurate
- **Adjust `speech_rate`** (100-200) for TTS speed

### Troubleshooting:

- **"Recording not working"** → Check microphone permissions
- **"Whisper slow"** → Use `tiny` model or upgrade GPU
- **"TTS not speaking"** → Check speaker/audio output
- **"Can't hear"** → Increase TTS volume in TextToSpeech init

---

## Next: Phase 4 - Voice Activity Detection (VAD)

To make it truly seamless, we can add VAD to automatically detect when HR starts/stops speaking (no need to press ENTER). This requires:
- Silero VAD or WebRTC VAD
- Real-time audio stream processing
- Automatic silence detection

Would you like me to implement Phase 4? 🚀

---

## Phase 4: Voice Activity Detection (VAD)

Automatic speech detection - no need to press ENTER!

### 14. VAD Dependencies

In [43]:
# Install VAD if needed (already installed: silero-vad)
# !pip install torch torchaudio

import torch
import time
from collections import deque

print("✅ VAD dependencies loaded")

✅ VAD dependencies loaded


### 15. Voice Activity Detector

In [61]:
class VoiceActivityDetector:
    """
    Voice Activity Detection using Silero VAD
    Automatically detects when user starts and stops speaking
    """
    
    def __init__(self, sample_rate=16000, threshold=0.5, min_silence_duration=0.5):
        """
        Initialize VAD with Silero model
        
        Args:
            sample_rate: Audio sample rate (must be 8000 or 16000)
            threshold: Voice probability threshold (0.0-1.0)
            min_silence_duration: Minimum silence duration to stop recording (seconds)
        """
        print("Loading Silero VAD model...")
        self.sample_rate = sample_rate
        self.threshold = threshold
        self.min_silence_duration = min_silence_duration
        
        # Silero VAD expects exactly 512 samples for 16kHz or 256 for 8kHz
        self.vad_chunk_size = 512 if sample_rate == 16000 else 256
        self.chunk_duration = self.vad_chunk_size / sample_rate  # ~0.032s for 16kHz
        
        # Load Silero VAD model
        self.model, utils = torch.hub.load(
            repo_or_dir='snakers4/silero-vad',
            model='silero_vad',
            force_reload=False,
            onnx=False
        )
        
        self.get_speech_timestamps = utils[0]
        print(f"✅ Silero VAD loaded (chunk size: {self.vad_chunk_size} samples)")
    
    def is_speech(self, audio_chunk):
        """
        Check if audio chunk contains speech
        
        Args:
            audio_chunk: numpy array of audio data (must be exactly 512 samples for 16kHz)
        
        Returns:
            Boolean indicating if speech is detected
        """
        # Ensure correct chunk size
        if len(audio_chunk) != self.vad_chunk_size:
            # Pad or truncate to correct size
            if len(audio_chunk) < self.vad_chunk_size:
                audio_chunk = np.pad(audio_chunk, (0, self.vad_chunk_size - len(audio_chunk)))
            else:
                audio_chunk = audio_chunk[:self.vad_chunk_size]
        
        # Convert to torch tensor
        audio_tensor = torch.from_numpy(audio_chunk).float()
        
        # Get speech probability
        speech_prob = self.model(audio_tensor, self.sample_rate).item()
        
        return speech_prob > self.threshold
    
    def record_until_silence(self, max_duration=30, manual_stop=True):
        """
        Record audio until silence is detected OR user presses ENTER
        Automatically starts when speech is detected, stops after silence or manual interrupt
        
        Args:
            max_duration: Maximum recording duration (seconds)
            manual_stop: If True, allow user to press ENTER to stop recording
        
        Returns:
            numpy array of recorded audio
        """
        import msvcrt  # For Windows keyboard input (use 'select' on Linux/Mac)
        import sys
        
        print("\n🎤 Listening... (speak when ready)")
        if manual_stop:
            print("💡 Press ENTER at any time to stop recording")
        
        # Use larger chunks for recording (e.g., 0.1s = 1600 samples)
        # Then process in VAD-sized chunks (512 samples)
        recording_chunk_duration = 0.1  # 100ms recording chunks
        recording_chunk_size = int(recording_chunk_duration * self.sample_rate)
        max_chunks = int(max_duration / recording_chunk_duration)
        
        audio_buffer = []
        speech_started = False
        silence_duration = 0.0
        manual_stopped = False
        
        for i in range(max_chunks):
            # Check for keyboard input (ENTER key)
            if manual_stop and msvcrt.kbhit():
                key = msvcrt.getch()
                if key in [b'\r', b'\n', b' ']:  # ENTER or SPACE
                    if speech_started:
                        print("⏹️ Manual stop - recording ended")
                        manual_stopped = True
                        break
                    else:
                        # Clear the buffer and continue listening
                        while msvcrt.kbhit():
                            msvcrt.getch()
            
            # Record chunk
            chunk = sd.rec(
                recording_chunk_size,
                samplerate=self.sample_rate,
                channels=1,
                dtype='float32'
            )
            sd.wait()
            chunk = chunk.flatten()
            
            # Process chunk in VAD-sized sub-chunks
            has_speech_in_chunk = False
            for j in range(0, len(chunk), self.vad_chunk_size):
                if j + self.vad_chunk_size <= len(chunk):
                    vad_chunk = chunk[j:j + self.vad_chunk_size]
                    if self.is_speech(vad_chunk):
                        has_speech_in_chunk = True
                        break
            
            if has_speech_in_chunk:
                if not speech_started:
                    print("🗣️ Speech detected! Recording...")
                    if manual_stop:
                        print("   (Press ENTER when done speaking)")
                    speech_started = True
                silence_duration = 0.0
                audio_buffer.append(chunk)
            elif speech_started:
                # Speech was happening, now silence
                silence_duration += recording_chunk_duration
                audio_buffer.append(chunk)
                
                if silence_duration >= self.min_silence_duration:
                    print(f"✅ Silence detected ({silence_duration:.1f}s). Recording stopped.")
                    break
        
        if not speech_started:
            print("⚠️ No speech detected")
            return None
        
        # Concatenate all chunks
        audio = np.concatenate(audio_buffer)
        return audio

print("✅ VoiceActivityDetector class defined")

✅ VoiceActivityDetector class defined


### 16. Hands-Free Voice Agent (with VAD)

In [62]:
class HandsFreeVoiceAgent(VoiceEnabledAgent):
    """
    Hands-free voice agent with automatic speech detection
    Just speak - no need to press ENTER!
    """
    
    def __init__(self, context: CVScoringContext, company_name: str = None,
                 whisper_model="base", speech_rate=150, vad_threshold=0.5):
        """
        Initialize hands-free agent with VAD
        
        Args:
            context: CVScoringContext with CV and JD data
            company_name: Optional company name
            whisper_model: Whisper model size
            speech_rate: TTS speech rate
            vad_threshold: VAD sensitivity (0.3-0.7, lower=more sensitive)
        """
        # Initialize parent voice agent
        super().__init__(context, company_name, whisper_model, speech_rate)
        
        # Initialize VAD
        print("\n🎙️ Initializing Voice Activity Detection...")
        self.vad = VoiceActivityDetector(
            sample_rate=self.stt.sample_rate,
            threshold=vad_threshold,
            min_silence_duration=0.8  # Stop after 0.8s of silence
        )
        
        print("✅ Hands-free agent ready!")
    
    def hands_free_chat(self, max_question_duration=30):
        """
        Fully hands-free voice conversation
        Just speak when ready, agent automatically detects speech
        
        Args:
            max_question_duration: Maximum duration for each question (seconds)
        """
        print("\n" + "="*60)
        print("🎤 HANDS-FREE VOICE AGENT")
        print("="*60)
        
        summary = self.context.get_summary()
        print(f"\nCandidate: {summary['candidate_name']}")
        print(f"Job Title: {summary['job_title']}")
        if summary['company']:
            print(f"Company: {summary['company']}")
        
        # Welcome message
        welcome = f"Hello! I'm ready to answer questions about {summary['candidate_name']}'s application. Just start speaking when you're ready!"
        print(f"\n🤖 Agent: {welcome}")
        self.tts.speak(welcome)
        
        print("\n" + "-"*60)
        print("Instructions:")
        print("- 🎤 Just start speaking (automatic detection)")
        print("- Agent stops recording after silence")
        print("- Say 'goodbye' or 'exit' to end session")
        print("- Press Ctrl+C to force quit")
        print("-"*60)
        
        conversation_count = 0
        
        while True:
            try:
                print(f"\n[Question #{conversation_count + 1}]")
                
                # Automatically record when speech is detected
                audio = self.vad.record_until_silence(max_duration=max_question_duration)
                
                if audio is None:
                    print("⚠️ No speech detected. Listening again...")
                    continue
                
                # Transcribe
                print("🔄 Transcribing...")
                question = self.stt.transcribe_audio(audio)
                
                # Check for exit commands
                if any(word in question.lower() for word in ['goodbye', 'quit', 'exit', 'bye', 'stop']):
                    goodbye = "Goodbye! Have a great day!"
                    print(f"\n🤖 Agent: {goodbye}")
                    self.tts.speak(goodbye)
                    break
                
                if not question or len(question.strip()) < 3:
                    prompt = "I didn't catch that clearly. Please try again."
                    print(f"\n🤖 Agent: {prompt}")
                    self.tts.speak(prompt)
                    continue
                
                # Display question
                print(f"\n🎤 HR: {question}")
                
                # Get answer from RAG
                print("🤖 Agent: ", end="", flush=True)
                result = self.ask(question)
                answer = result["answer"]
                
                print(answer)
                
                # Speak answer
                self.tts.speak(answer)
                
                # Show metadata
                if result["sources"]:
                    print(f"\n📚 Retrieved {len(result['sources'])} source(s)")
                
                conversation_count += 1
                
                # Brief pause before next question
                time.sleep(0.5)
                print("\n" + "-"*60)
                print("Ready for next question...")
            
            except KeyboardInterrupt:
                print("\n\n⚠️ Session interrupted")
                goodbye = "Session ended. Goodbye!"
                print(f"\n🤖 Agent: {goodbye}")
                self.tts.speak(goodbye)
                break
            
            except Exception as e:
                error_msg = f"An error occurred: {str(e)}"
                print(f"\n❌ {error_msg}")
                self.tts.speak("I encountered an error. Please try again.")
                continue

print("✅ HandsFreeVoiceAgent class defined")

✅ HandsFreeVoiceAgent class defined


### 17. Initialize Hands-Free Agent

In [63]:
# Initialize the hands-free voice agent with existing context
print("Initializing Hands-Free Voice Agent...")
print(f"CV: {context.cv_id}")
print(f"JD: {context.jd_id}")

# Create hands-free agent with moderate VAD sensitivity
# Lower threshold (0.3-0.4) = more sensitive (picks up softer speech, may trigger on noise)
# Higher threshold (0.6-0.7) = less sensitive (requires clearer speech, may miss soft words)
# Recommended: 0.5 for balanced performance
handsfree_agent = HandsFreeVoiceAgent(
    context=context,
    company_name=company_name,
    whisper_model="base",
    speech_rate=150,
    vad_threshold=0.5
)

print("\n✅ Hands-Free Voice Agent ready!")
print("📌 No manual triggers needed - the agent will automatically detect when you speak")
print("📌 Just speak naturally and the agent will respond")

Initializing Hands-Free Voice Agent...
CV: 2e538000bef0ba2c6bfd10f0fb99b0d97843da9e35f46b255c59141bc3660484
JD: 22c009485a6d8f139582719426054c126f7f8b426351dbfb5681cddb42ae180d
✅ Connected to ChromaDB: cv_sections
✅ QA Chain initialized

🎙️ Initializing voice capabilities...
Loading Whisper model: base...
✅ Whisper base model loaded
✅ TTS engine initialized
✅ Voice-enabled agent ready!

🎙️ Initializing Voice Activity Detection...
Loading Silero VAD model...
✅ Silero VAD loaded (chunk size: 512 samples)
✅ Hands-free agent ready!

✅ Hands-Free Voice Agent ready!
📌 No manual triggers needed - the agent will automatically detect when you speak
📌 Just speak naturally and the agent will respond


Using cache found in C:\Users\Enoch/.cache\torch\hub\snakers4_silero-vad_master


### Quick VAD Test

Let's test the VAD in isolation to ensure it works correctly:

In [64]:
# Quick test of VAD functionality
print("Testing VAD setup...")

# Create test VAD instance
test_vad = VoiceActivityDetector(sample_rate=16000, threshold=0.5, min_silence_duration=0.8)

# Create a test audio chunk of the correct size
test_chunk = np.random.randn(512).astype('float32')
print(f"Test chunk size: {len(test_chunk)} samples")

# Test is_speech function
try:
    result = test_vad.is_speech(test_chunk)
    print(f"✅ VAD test passed! Speech detected: {result}")
    print(f"VAD chunk size: {test_vad.vad_chunk_size}")
    print(f"Recording chunk size: {int(0.1 * test_vad.sample_rate)} samples")
except Exception as e:
    print(f"❌ VAD test failed: {e}")

Testing VAD setup...
Loading Silero VAD model...
✅ Silero VAD loaded (chunk size: 512 samples)
Test chunk size: 512 samples
✅ VAD test passed! Speech detected: False
VAD chunk size: 512
Recording chunk size: 1600 samples


Using cache found in C:\Users\Enoch/.cache\torch\hub\snakers4_silero-vad_master


### 18. Start Hands-Free Conversation

**🎤 Usage Instructions:**
- Just run the cell below and start speaking when ready
- No need to press any keys - the agent automatically detects your voice
- Speak naturally and wait for the agent to respond
- Say "goodbye", "quit", or "exit" to end the conversation
- Press Ctrl+C to force quit if needed

**⚙️ VAD Threshold Tuning:**
- Current: 0.5 (balanced)
- If agent doesn't detect your voice: Lower to 0.3-0.4
- If agent triggers on background noise: Raise to 0.6-0.7
- Adjust in the initialization cell above and re-run

In [66]:
# Start hands-free conversation
# The agent will automatically listen for your voice and respond
# No manual triggers needed - just speak!
handsfree_agent.hands_free_chat()


🎤 HANDS-FREE VOICE AGENT

Candidate: Enoch Kwadwo Aidoo
Job Title: Data Analyst
Company: N/A

🤖 Agent: Hello! I'm ready to answer questions about Enoch Kwadwo Aidoo's application. Just start speaking when you're ready!
🔊 Speaking: "Hello! I'm ready to answer questions about Enoch K..."

------------------------------------------------------------
Instructions:
- 🎤 Just start speaking (automatic detection)
- Agent stops recording after silence
- Say 'goodbye' or 'exit' to end session
- Press Ctrl+C to force quit
------------------------------------------------------------

[Question #1]

🎤 Listening... (speak when ready)
💡 Press ENTER at any time to stop recording
🗣️ Speech detected! Recording...
   (Press ENTER when done speaking)
🗣️ Speech detected! Recording...
   (Press ENTER when done speaking)
✅ Silence detected (0.9s). Recording stopped.
🔄 Transcribing...
🔄 Transcribing...
✅ Silence detected (0.9s). Recording stopped.
🔄 Transcribing...
🔄 Transcribing...
📝 Transcribed: "Hello, he